In [2]:
import time
from rdkit import Chem
from rdkit.Chem import AllChem, TorsionFingerprints
import json
import tqdm
import numpy as np

from utils import *

/home/tgog/.conda/envs/my-rdkit-env/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


_ColormakerRegistry()

In [3]:
def create_branched():
    m = Chem.MolFromSmiles('CCCC')
    e = Chem.RWMol(m)

    numatoms = len(e.GetAtoms())
    tot = np.random.choice([15,16,17,18,19,20])
    while numatoms < tot:
        x = Chem.rdchem.Atom(6)
        randidx = np.random.randint(len(e.GetAtoms()))
        atom = e.GetAtomWithIdx(randidx)
        if atom.GetDegree() > 2:
            continue
        idx = e.AddAtom(x)
        e.AddBond(idx, randidx, Chem.rdchem.BondType.SINGLE)
        numatoms = len(e.GetAtoms())


    Chem.SanitizeMol(e)
    m = Chem.rdmolops.AddHs(e.GetMol())
    AllChem.EmbedMultipleConfs(m, numConfs=200, numThreads=-1)
    Chem.AllChem.MMFFOptimizeMoleculeConfs(m, numThreads=-1)


    confgen = ConformerGeneratorCustom(max_conformers=1, 
                     rmsd_threshold=None, 
                     force_field='mmff',
                     pool_multiplier=1)  

    energys = confgen.get_conformer_energies(m)
    print(len(TorsionFingerprints.CalculateTorsionLists(m)[0]))
    standard = energys.min()
    total = np.sum(np.exp(-(energys-standard)))
    
    out = {
        'mol': Chem.MolToSmiles(m, isomericSmiles=False),
        'standard': standard,
        'total': total
    }
    return out

In [ ]:
for i in tqdm(range(500)):
    out = create_branched()
    with open(f'bigger_labeled/{i}.json', 'w') as fp:
        json.dump(out, fp)



  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 1/500 [00:07<1:03:41,  7.66s/it]

8




  0%|          | 2/500 [00:17<1:09:22,  8.36s/it]

11




  1%|          | 3/500 [00:26<1:09:30,  8.39s/it]

...

10




 97%|█████████▋| 484/500 [1:07:59<02:15,  8.49s/it]

7




 97%|█████████▋| 485/500 [1:08:09<02:15,  9.02s/it]

10


In [ ]:
create_branched()

In [56]:
confgen = ConformerGeneratorCustom(max_conformers=1, 
                 rmsd_threshold=None, 
                 force_field='mmff',
                 pool_multiplier=1)
x = Chem.MolFromSmiles('[H]C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])(C([H])([H])[H])C([H])(C([H])([H])C([H])([H])C([H])([H])[H])C([H])(C([H])([H])C([H])([H])[H])C([H])([H])C([H])([H])[H]')
res = AllChem.EmbedMultipleConfs(x, numConfs=1, numThreads=-1)
confgen.get_conformer_energies(x).min()
print(res)
print(type(res))
print(len(res))

<class 'rdkit.rdBase._vecti'>
1


In [57]:
import subprocess
subprocess.check_output("mkdir bigger_labeled", shell=True)

b''

In [ ]:
from glob import glob

glob('bigger_branched/*')

In [44]:
obj = json.load(open('huge_hc_set/10_8.json'))
mol = obj['mol']
stand = obj['standard']
x = Chem.MolFromSmiles(mol)
mol = Chem.AddHs(x)
AllChem.EmbedMultipleConfs(mol, numConfs=200, numThreads=-1)
Chem.AllChem.MMFFOptimizeMoleculeConfs(mol, numThreads=-1)

confgen = ConformerGeneratorCustom(max_conformers=200, 
                 rmsd_threshold=1.0, 
                 force_field='mmff',
                 pool_multiplier=1)  

# mol = Chem.RemoveHs(mol)
# mol, mat = confgen.prune_conformers(mol, get_conformer_rmsd_fast(mol))
# mol = Chem.AddHs(mol)

energys = confgen.get_conformer_energies(mol)
print(len(energys))

200


In [45]:
np.sum(np.exp( -1.0*(energys - stand)[energys - stand < 6]))

7.878298354122746

In [ ]:
stand

In [ ]:
x= mol.GetConformer(-1)
x.GetPositions()

In [ ]:
mol.GetNumAtoms()

In [ ]:
mol, mat = confgen.prune_conformers(mol, get_conformer_rmsd_fast(mol, heavy_atoms_only=True), heavy_atoms_only=True )
print(mat)

In [ ]:
nonring, ring = Chem.TorsionFingerprints.CalculateTorsionLists(mol)
all_tors = [Chem.TorsionFingerprints.CalculateTorsionAngles(mol, nonring, ring, confId=i) for i in range(200)]


In [52]:
out = array_to_lower_triangle(Chem.TorsionFingerprints.GetTFDMatrix(mol, useWeights=False))
print(out)

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.4251802  0.         0.         ... 0.         0.         0.        ]
 [0.37490555 0.44893665 0.         ... 0.         0.         0.        ]
 ...
 [0.38063435 0.25389999 0.5797362  ... 0.         0.         0.        ]
 [0.34415208 0.23391711 0.5190535  ... 0.3089227  0.         0.        ]
 [0.39750685 0.34495811 0.42938288 ... 0.27128223 0.49223518 0.        ]]


In [54]:
np.sum(np.logical_and(out>0.000, out<0.10))

3

In [65]:
hmm = np.logical_and(out>0.000, out<0.10)
for i in range(200):
    for j in range(200):
        if hmm[i][j] > 0:
            print(i, j)

138 34
144 95
177 0


In [17]:
np.any((out>0.000, out<0.01))

True

In [68]:
nonring, ring = Chem.TorsionFingerprints.CalculateTorsionLists(mol)
Chem.TorsionFingerprints.CalculateTorsionAngles(mol, nonring, ring, confId=177)

[([176.5118953638173], 180.0),
 ([198.89347146276245], 180.0),
 ([65.27336630307472], 180.0),
 ([135.38767128374582], 180.0),
 ([139.73627915406328], 180.0),
 ([189.14281883393562], 180.0),
 ([171.9235276080036], 180.0),
 ([183.1461051169328], 180.0),
 ([196.32094611374652], 180.0),
 ([317.4839087975374], 180.0)]

In [69]:
Chem.TorsionFingerprints.CalculateTorsionAngles(mol, nonring, ring, confId=0)

[([176.3867542120988], 180.0),
 ([198.65929435741808], 180.0),
 ([65.10711044054644], 180.0),
 ([134.51851418282163], 180.0),
 ([140.18580351226126], 180.0),
 ([188.06184242779867], 180.0),
 ([172.10194113202803], 180.0),
 ([181.89201673283483], 180.0),
 ([69.91554918211719], 180.0),
 ([292.8514970177645], 180.0)]

In [12]:
# mol = Chem.MolFromSmiles('CC(CCC)CCCC(CCCC)CC')
# standard = 7.678142433712216

mol = Chem.MolFromMolFile('258-trihexyl-decane.mol')
standard = 14.88278294332602
mol = Chem.AddHs(mol)
res = AllChem.EmbedMultipleConfs(mol, numConfs=200, numThreads=-1)
print(res[0])
res = AllChem.MMFFOptimizeMoleculeConfs(mol, numThreads=-1)
print(res)

0
[(1, 22.702735114921346), (1, 23.226607481430296), (1, 16.217599875583346), (1, 20.084658722819963), (1, 23.355175381128337), (1, 23.821319529384194), (1, 32.624193665607926), (1, 26.43955845089064), (1, 25.440372751680947), (1, 26.662578481148845), (1, 20.9097731025148), (1, 21.622563174923073), (1, 20.102477918417506), (1, 29.5811847010739), (1, 21.69803949759398), (1, 22.221746440745882), (1, 20.78079491325065), (1, 22.936405860922157), (1, 23.296734414644337), (1, 29.788142319260377), (1, 26.286752556448853), (1, 27.447635416599184), (1, 20.461050576936916), (1, 30.487101049565545), (1, 26.8815021868532), (1, 21.35945493863967), (1, 25.663233944824928), (1, 24.626283019582406), (1, 22.09984133559549), (1, 17.56061750654976), (1, 23.459435163602606), (1, 19.26960350536428), (1, 28.714995799709236), (1, 24.45981636246118), (1, 26.341129098552617), (1, 26.344226753411856), (1, 19.89699009066934), (1, 29.81961616331778), (1, 30.679038750500357), (1, 28.238605476716355), (1, 26.578717

In [7]:
confgen = ConformerGeneratorCustom(max_conformers=200, 
                 rmsd_threshold=1.0, 
                 force_field='mmff',
                 pool_multiplier=1)  
energys = confgen.get_conformer_energies(mol)

In [14]:
np.sum(np.exp(-(energys - standard)))

2.343469632551261

In [15]:
out = array_to_lower_triangle(Chem.TorsionFingerprints.GetTFDMatrix(mol, useWeights=True))
out

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.22070328, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.24094625, 0.14549333, 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.14232052, 0.20321054, 0.16155836, ..., 0.        , 0.        ,
        0.        ],
       [0.18807089, 0.22388068, 0.2580879 , ..., 0.16597064, 0.        ,
        0.        ],
       [0.19506699, 0.12048977, 0.19172023, ..., 0.2493807 , 0.25849995,
        0.        ]])

In [30]:
np.any((out>0.000, out<0.01))

True

In [15]:
def prune_conformers(mol, tfd_thresh):
    """
    Prune conformers from a molecule using an RMSD threshold, starting
    with the lowest energy conformer.

    Parameters
    ----------
    mol : RDKit Mol
            Molecule.

    Returns
    -------
    new: A new RDKit Mol containing the chosen conformers, sorted by
             increasing energy.
    new_rmsd: matrix of conformer-conformer RMSD
    """
    if tfd_thresh < 0 or mol.GetNumConformers() <= 1:
        return mol
    
    energies = confgen.get_conformer_energies(mol)
    
    tfd = array_to_lower_triangle(Chem.TorsionFingerprints.GetTFDMatrix(mol, useWeights=False), True)
    print(tfd)
    sort = np.argsort(energies)  # sort by increasing energy
    keep = []  # always keep lowest-energy conformer
    discard = []

    for i in sort:
        # always keep lowest-energy conformer
        if len(keep) == 0:
            keep.append(i)
            continue

        # get RMSD to selected conformers
        this_tfd = tfd[i][np.asarray(keep, dtype=int)]
        # discard conformers within the RMSD threshold
        if np.all(this_tfd >= tfd_thresh):
            keep.append(i)
        else:
            discard.append(i)

    # create a new molecule to hold the chosen conformers
    # this ensures proper conformer IDs and energy-based ordering
    new = Chem.Mol(mol)
    new.RemoveAllConformers()
    conf_ids = [conf.GetId() for conf in mol.GetConformers()]
    for i in keep:
        conf = mol.GetConformer(conf_ids[i])
        new.AddConformer(conf, assignId=True)
        
    return new

In [64]:
new = prune_conformers(mol, 0.05)

In [65]:
len(new.GetConformers())

200

In [28]:
np.sum(np.exp(-(energys - standard)[110]))

7.201980321773156e-05

In [35]:
nonring, ring = Chem.TorsionFingerprints.CalculateTorsionLists(mol)
len(nonring)

22

In [22]:
mol = Chem.MolFromSmiles('CC(CCC)CCCC(CCCC)CC')
mol = Chem.AddHs(mol)
nonring, ring = Chem.TorsionFingerprints.CalculateTorsionLists(mol)
AllChem.EmbedMultipleConfs(mol, numConfs=200, numThreads=-1)
Chem.AllChem.MMFFOptimizeMoleculeConfs(mol, numThreads=-1)
mol = prune_conformers(mol, 0.05)
len(nonring)


[[0.         0.4499015  0.25569097 ... 0.5425189  0.46507942 0.39047858]
 [0.4499015  0.         0.35404458 ... 0.40674323 0.41126591 0.49205703]
 [0.25569097 0.35404458 0.         ... 0.37494808 0.51665232 0.39194451]
 ...
 [0.5425189  0.40674323 0.37494808 ... 0.         0.3084418  0.3550111 ]
 [0.46507942 0.41126591 0.51665232 ... 0.3084418  0.         0.23580212]
 [0.39047858 0.49205703 0.39194451 ... 0.3550111  0.23580212 0.        ]]


10

In [46]:
import py3Dmol
p = py3Dmol.view(width=800,height=800)
drawit(mol, p, confId=0)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [24]:
energys = confgen.get_conformer_energies(mol)
standard = energys.min()
total = np.sum(np.exp(-(energys-standard)))
print(total)

13.263723987526067


In [25]:
standard

7.668625034772399

In [26]:
len(energys)

197

In [27]:
energys

array([ 7.66862503,  8.10671589,  8.32337611,  8.33990446,  8.38171277,
        8.46286725,  8.50308715,  8.56816811,  8.57750742,  8.5799142 ,
        8.64984953,  8.66494125,  8.79198307,  8.8046882 ,  8.90315019,
        8.93706831,  8.96130887,  8.97694614,  9.00393411,  9.09615244,
        9.12222597,  9.12711338,  9.23096527,  9.3189309 ,  9.34057775,
        9.44988819,  9.47437753,  9.48247121,  9.49885399,  9.50608707,
        9.52853849,  9.5517979 ,  9.66841597,  9.7250019 ,  9.74068695,
        9.83641345,  9.84032427,  9.85767777,  9.93739083, 10.1216748 ,
       10.16109733, 10.22353394, 10.33737435, 10.51364976, 10.56586696,
       10.73463717, 10.80996971, 10.88636186, 10.89769897, 10.96160705,
       10.97215261, 11.00365343, 11.09314378, 11.14818879, 11.17500875,
       11.22124203, 11.26090662, 11.2746021 , 11.27999457, 11.28751589,
       11.29126991, 11.30916588, 11.31881751, 11.36115305, 11.36981379,
       11.40053498, 11.44697088, 11.47379281, 11.47442655, 11.47

In [42]:
np.sum(np.exp(-(energys[-1]-standard)))

7.300408936747106e-08